<a href="https://colab.research.google.com/github/harryguiacorn/colab/blob/main/Generate_1hr_cloud_signal_and_charts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance ta pandas matplotlib mpl_finance mplfinance

# Download 1h data and generator cloud signals

In [ ]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
import mplfinance as mpf
from ta.trend import IchimokuIndicator

class IchimokuAnalyzer:
    def __init__(self, symbol, period, interval, output_folder):
        self.symbol = symbol
        self.period = period
        self.interval = interval
        self.output_folder = output_folder

    def create_folder(self, folder_path):
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    def download_and_analyze_data(self):
        # Create folders for CSV and image files
        folder_path_csv = os.path.join(self.output_folder, 'csv', self.symbol)
        folder_path_img = os.path.join(self.output_folder, 'img', self.symbol)

        self.create_folder(folder_path_csv)
        self.create_folder(folder_path_img)

        # Fetch hourly data for the specified period
        data = yf.download(self.symbol, period=self.period, interval=self.interval)

        # Create an IchimokuIndicator instance
        ichimoku = IchimokuIndicator(data['High'], data['Low'])

        # Calculate Ichimoku values
        data['Ichimoku_A'] = ichimoku.ichimoku_a().shift(26)
        data['Ichimoku_B'] = ichimoku.ichimoku_b().shift(26)

        # Create the 'Cloud' column based on Ichimoku values
        data['Cloud'] = np.where(
            (data['Close'] > data['Ichimoku_A']) & (data['Close'] > data['Ichimoku_B']), 1,
            np.where((data['Close'] < data['Ichimoku_A']) & (data['Close'] < data['Ichimoku_B']), -1, 0)
        )

        # Export data to a CSV file
        csv_filename = os.path.join(folder_path_csv, f'{self.symbol}_hourly_cloud_data_{self.period}.csv')
        data.to_csv(csv_filename)
        print(f"Hourly data for {self.symbol} over the last {self.period} with Ichimoku cloud values exported to {csv_filename}")

        # Filter the data for the specified time range (09:00 - 11:00)
        filtered_data = data.between_time('04:00', '15:00')  # EST time

        # Create and save candlestick charts for each day in the specified time range
        for date, group in filtered_data.groupby(filtered_data.index.date):
            date_str = date.strftime('%Y-%m-%d')
            time_str = group.index[0].strftime('%H-%M')
            chart_filename = f'{self.symbol}_{date_str}_{time_str}_candlestick_chart.png'
            chart_path = os.path.join(folder_path_img, chart_filename)
            mpf.plot(group, type='candle', savefig=chart_path, title=f'{self.symbol} {date} {self.interval} Candlestick Chart', style='yahoo')
            print(f"Candlestick chart for {self.symbol} on {date} saved as {chart_path}")

if __name__ == "__main__":
    symbol = 'ES=F'
    period = '6mo'
    interval = '1h'
    output_folder = 'output'

    analyzer = IchimokuAnalyzer(symbol, period, interval, output_folder)
    analyzer.download_and_analyze_data()


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates

# Define the stock symbol (AAPL for Apple Inc.)
stock_symbol = "JPY=X"

# Define the date range for the data
start_date = "2023-01-01"
end_date = "2023-12-31"

# Download the data from Yahoo Finance
data = yf.download(stock_symbol, start=start_date, end=end_date)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Resample the data to weekly OHLC
ohlc_data = data['Adj Close'].resample('W').ohlc()
ohlc_data.reset_index(inplace=True)
ohlc_data['Date'] = ohlc_data['Date'].map(mdates.date2num)

# Plot the Candlestick Chart
candlestick_ohlc(ax, ohlc_data.values, width=0.5, colorup='g', colordown='r')

# Add labels and title
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.set_title('AAPL Weekly Candlestick Chart')

# Format the x-axis to show dates nicely
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.grid()
plt.tight_layout()
plt.show()
